In [7]:
from datetime import datetime
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.service import Service
import time
import sys


#!!! CHANGE THE PATH FOR THE WEBDRIVER HERE!!!#
PATH_WEBDRIVER = 'C:/Users/Samsung/Documents/GitHub-Datas/chromedriver-win64/chromedriver.exe'

LINK = 'https://www.b3.com.br/pt_br/produtos-e-servicos/negociacao/renda-variavel/empresas-listadas.htm'

#####################       Functions        #########################

def read_tickers(driver, tickers):

    blocks = driver.find_elements(By.XPATH, '//*[@id="nav-bloco"]/div/div')

    # Cria uma lista temporária para armazenar as novas linhas
    new_rows = []

    for i in blocks:
        new_rows.append(i.text.splitlines())

    # Usa pd.concat() para adicionar as novas linhas ao DataFrame
    tickers = pd.concat([tickers, pd.DataFrame(new_rows)], ignore_index=True)

    return tickers

def next_pag(driver):
    driver.execute_script("window.scrollTo(0, 9000)")

    time.sleep(3) 

    driver.execute_script("window.scrollTo(0, 9000)") 

    driver.find_element(By.XPATH, '//*[@id="listing_pagination"]/pagination-template/ul/li[10]/a').click()


#####################       main        #########################

if __name__ == '__main__':

    # Set up the WebDriver using Service
    try:
        service = Service(PATH_WEBDRIVER)
        driver = webdriver.Chrome(service=service)
    except:
        print('Error opening webdriver')
        sys.exit(1)

    # Open the page
    driver.get(LINK)

    timeout = 10

    # Get cookies out of the way
    try:
        WebDriverWait(driver, timeout).until(EC.element_to_be_clickable((By.XPATH, "//*[@id=\"onetrust-accept-btn-handler\"]")))
    except TimeoutException:
        print('Timeout occurred when waiting for cookies popup')

    driver.find_element(By.XPATH, '//*[@id=\"onetrust-accept-btn-handler\"]').click()

    # Wait until 'Todos' appears and click on it
    try:
        WebDriverWait(driver, timeout).until(EC.visibility_of_element_located((By.XPATH, "//*[@id=\"bvmf_iframe\"]")))
    except TimeoutException:
        print('Timeout occurred waiting for iframe')

    driver.execute_script("window.scrollTo(0, 250)")
    iframe = driver.find_element(By.XPATH, "//*[@id=\"bvmf_iframe\"]")
    driver.switch_to.frame(iframe)

    try:
        WebDriverWait(driver, timeout).until(EC.element_to_be_clickable((By.XPATH, '//button[text()="Todos"]')))
    except TimeoutException:
        print('Timeout occurred waiting for Todos button')

    driver.find_element(By.XPATH, '//button[text()="Todos"]').click()

    print('Page 1')

    # Wait until you can select to show 120 tickers per page
    try:
        WebDriverWait(driver, timeout).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="selectPage"]')))
    except TimeoutException:
        print('Timeout occurred waiting for selectPage dropdown')

    Select(driver.find_element(By.XPATH, '//*[@id="selectPage"]')).select_by_visible_text('120')

    time.sleep(3)

    # Create dataframe to store the tickers
    tickers = pd.DataFrame()

    # Get the first tickers
    tickers = read_tickers(driver, tickers)

    # While there are pages, keep scraping
    pg = 2

    while True:
        try:
            next_pag(driver)
            print('Page ', pg)
            time.sleep(3)
            tickers = read_tickers(driver, tickers)
            pg += 1
        except:
            print("Last page done")
            break

    # Exit driver
    driver.quit()

    # Clean and save the data
    print("Saving data")
    tickers = tickers.rename(columns={0:'Code', 1:'Nome', 2:'Nome fantasia', 3:'Governança'})
    tickers['Governança'] = tickers['Governança'].fillna('-')

    name = 'B3_companies_' + datetime.today().strftime('%Y-%m-%d') + '.csv'
    tickers.to_csv(name, index=False)


Page 1
Page  2
Page  3
Page  4
Page  5
Page  6
Page  7
Page  8
Page  9
Page  10
Page  11
Page  12
Page  13
Page  14
Page  15
Page  16
Page  17
Page  18
Page  19
Page  20
Page  21
Page  22
Page  23
Page  24
Last page done
Saving data
